# Data Analysis and Preparation
**Author: Tesfagabir Meharizghi<br>
Last Updated: 02/12/2021**

This Notebook does the following actions to the one-month original data and the balanced all-months data:
- Reads the flattened data
- Removes empty rows
- Adds information of patients having at least one Ground Truth Codes
- Splits Data into Train/Val/Test
- Saves data

### Final Analysis:
+ Available GT codes original: 134

+ 1 Month Data (20110101)
    + Seq Length: 1000
    + Total rows: ~1.7M (1657688, 1002)
    + Total vocab: 1st Month flattened: 33K (33255)
    + Intersection: 89
    + Total Patients with GT Codes: 841340
        - Positive: 295
        - Negative: 838391
+ All Downsampled Data (Balanced)
    + Seq Length: 1000
    + Total rows: ~22K (21981, 1002)
    + Total vocab: 20K (21856)
    + Intersection ground truth: 83
    + Total Patients with GT Codes: 9450
        - Positive: 7635
        - Negative: 1815

In [3]:
%load_ext lab_black

%load_ext autoreload

%autoreload 2

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
SEQ_LEN = 1000
TEST_SIZE = 0.15

ALL_DOWN_DATA_PATH = f"./output/data/1000/downsampled/all.csv"

MONTH_DATA_PATH = f"./output/data/1000/original/20110101.csv"

GT_CODES_PATH = "./cdiff_risk_factors_codes.csv"

ALL_DOWN_DATA_OUT_PATH = f"./output/data/1000/downsampled/preprocessed/all.csv"

MONTH_DATA_OUT_PATH = f"./output/data/1000/original/preprocessed/20110101.csv"

In [9]:
# train_path = "./output/data/1000/downsampled/preprocessed/splits/all/train.csv"
# out_path = "./output/data/1000/downsampled/preprocessed/splits/all/train2.csv"

train_path = "./output/data/1000/original/preprocessed/splits/20110101/train.csv"
out_path = "./output/data/1000/original//preprocessed/splits/20110101//train2.csv"

df_train = pd.read_csv(train_path)
df_train.head()

,999,998,997,996,995,994,993,992,991,990,...,5,4,3,2,1,0,d_00845,patient_id,num_gt_codes,has_gt_codes
0,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_G0103,7_days,d_s4011,d_sV7641,h_82270,h_99215,0,KSIDQZUKM,0,0
1,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_92015,h_92250,46_days,d_sV0481,h_90658,h_G0008,0,SDZPYF21D,0,0
2,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,<pad>,<pad>,<pad>,<pad>,d_s4139,h_99214,0,IJ5K5BJ4Z,0,0
3,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_s7390,d_s7394,h_98940,63_days,d_s7842,h_99203,0,CNRNLCMKD,0,0
4,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_83036,h_83615,h_85025,h_90732,h_93000,h_99214,0,TL9W8N0VK,0,0


In [10]:
df_train = df_train.sort_values("has_gt_codes", ascending=False)
df_train.head()

,999,998,997,996,995,994,993,992,991,990,...,5,4,3,2,1,0,d_00845,patient_id,num_gt_codes,has_gt_codes
580190,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_A4253,h_A4259,p_D1E,12_days,d_s4011,h_99214,0,5GUHQ99SV,1,1
669505,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_s3320,h_99213,14_days,d_s49120,h_E0570,p_D1E,0,JOPIMUTW6,2,1
242568,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_E1390,p_D1C,2_days,d_sV431,h_92014,h_92015,0,NXXWG73Q3,2,1
669522,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_s2449,d_s2724,d_s56409,h_80053,h_80061,h_84439,0,BGB0PGQNR,1,1
242576,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_A4259,p_D1E,15_days,d_25000,h_A4258,h_E0607,0,NGQU7JCQH,1,1


In [ ]:
df_train.to_csv(out_path, index=False)

In [47]:
def get_gt_code_patient(row, gt_codes, seq_len=1000):
    """Get patient_id of those having the gt_codes"""
    cols = [str(i) for i in range(seq_len - 1, -1, -1)]
    num_common = len(set(row[cols]).intersection(gt_codes))
    return num_common

def split_data_v2(df, test_size, label, output_dir, n_events=1000):
    """Split data into train/val/test sets. test_size is the fraction of val/test sets"""
    df['has_gt_codes'] = (df['num_gt_codes']>0).astype(int)
    stratify_cols = [label, 'has_gt_codes']
    df_train, df_val_test = train_test_split(
            df,
            test_size=2 * test_size,
            stratify=df[stratify_cols])

    df_val, df_test = train_test_split(
            df_val_test,
            test_size=0.5,
            stratify=df_val_test[stratify_cols])
    
    os.makedirs(output_dir, exist_ok=True)
    
    #Sort val & test
    df_val = df_val.sort_values('has_gt_codes', ascending=False)
    df_test = df_test.sort_values('has_gt_codes', ascending=False)
    
    df_train.to_csv(os.path.join(output_dir, "train.csv"), index=False)
    df_val.to_csv(os.path.join(output_dir, "val.csv"), index=False)
    df_test.to_csv(os.path.join(output_dir, "test.csv"), index=False)
    return df_train, df_val, df_test

In [ ]:
#Read GT Codes
gt_codes = pd.read_csv(GT_CODES_PATH)
gt_codes = set(gt_codes.Internal_Code)
print('Total GT Codes:', len(gt_codes))

df = pd.read_csv(ALL_DOWN_DATA_PATH)
print("Original shape:", df.shape)
# Remove empty rows
df = df[df["0"] != "<pad>"]
print("Without Empty shape:", df.shape)

df["num_gt_codes"] = df.apply(get_gt_code_patient, args=(gt_codes, SEQ_LEN), axis=1)
print("Total_patients with GT Codes:", sum(df["num_gt_codes"] > 0))

print(df[df["num_gt_codes"] != 0]["d_00845"].value_counts())
df[df["num_gt_codes"] != 0]["d_00845"].value_counts().plot.bar()

output_dir = os.path.dirname(ALL_DOWN_DATA_OUT_PATH)
os.makedirs(output_dir, exist_ok=True)
df.to_csv(ALL_DOWN_DATA_OUT_PATH, index=False)

print('Spliting data...')
output_dir, fname = ALL_DOWN_DATA_OUT_PATH.rsplit('/', 1)
fname = fname.split('.')[0]
output_dir = os.path.join(output_dir, 'splits', fname)
_ = split_data_v2(df, TEST_SIZE, LABEL, output_dir)

print(f'Preprocessed data successfully saved to {ALL_DOWN_DATA_OUT_PATH}')
print('-'*30)

df = pd.read_csv(MONTH_DATA_PATH)
print("Original shape:", df.shape)
# Remove empty rows
df = df[df["0"] != "<pad>"]
print("Without Empty shape:", df.shape)

df["num_gt_codes"] = df.apply(get_gt_code_patient, args=(gt_codes, SEQ_LEN), axis=1)
print("Total_patients with GT Codes:", sum(df["num_gt_codes"] > 0))

print(df[df["num_gt_codes"] != 0]["d_00845"].value_counts())
df[df["num_gt_codes"] != 0]["d_00845"].value_counts().plot.bar()

output_dir = os.path.dirname(MONTH_DATA_OUT_PATH)
os.makedirs(output_dir, exist_ok=True)
df.to_csv(MONTH_DATA_OUT_PATH, index=False)

print('Spliting data...')
output_dir, fname = MONTH_DATA_OUT_PATH.rsplit('/', 1)
fname = fname.split('.')[0]
output_dir = os.path.join(output_dir, 'splits', fname)
_ = split_data_v2(df, TEST_SIZE, LABEL, output_dir)
print(f'Preprocessed data successfully saved to {MONTH_DATA_OUT_PATH}')